In [ ]:
        # # Calculate the difference in coordinates before and after the zoom
        # delta_x = (x - self.canvas.canvasx(event.x)) * factor
        # delta_y = (y - self.canvas.canvasy(event.y)) * factor
        
        # # Adjust the scroll region of the canvas to keep the mouse cursor fixed
        # self.canvas.xview_scroll(int(delta_x), "units")
        # self.canvas.yview_scroll(int(delta_y), "units")

        # Adjust the scroll region of the canvas
        # new_width = int(self.canvas.winfo_width() * factor)
        # new_height = int(self.canvas.winfo_height() * factor)
        # new_x = x - (x - self.canvas.xview()[0] * self.canvas.winfo_width()) * factor
        # new_y = y - (y - self.canvas.yview()[0] * self.canvas.winfo_height()) * factor
        # self.canvas.configure(scrollregion=(0, 0, new_width, new_height))
        # self.canvas.xview_moveto(new_x / new_width)
        # self.canvas.yview_moveto(new_y / new_height)

In [ ]:
    # def zoom(self, event):
    #     cursor_x = self.canvas.canvasx(event.x)
    #     cursor_y = self.canvas.canvasy(event.y)
    #     factor = 1.1 if event.delta > 0 else 0.9
        
        
    #     # Adjust the offset based on the cursor position
    #     self.offset_x = cursor_x - factor * (cursor_x - self.offset_x)
    #     self.offset_y = cursor_y - factor * (cursor_y - self.offset_y)

    #     self.zoom_level *= factor
        
    #     self.update_image()


    # def pan_start(self, event):
    #     self.pan_x = event.x
    #     self.pan_y = event.y
    

    # def pan_left(self, event):
    #     #self.offset_x = max(0, self.offset_x -0.1 * self.canvas.winfo_width())
    #     self.offset_x =  self.offset_x -0.1 * self.canvas.winfo_width()
    #     self.update_image()

    # def pan_right(self, event):
    #     #self.offset_x = min(self.canvas.winfo_width(), self.offset_x +0.1 * self.canvas.winfo_width())
    #     self.offset_x = self.offset_x +0.1 * self.canvas.winfo_width()
    #     self.update_image()

    # def pan_down(self, event):
    #     #self.offset_y =max(-self.canvas.winfo_height(), self.offset_y -0.1 * self.canvas.winfo_height())
    #     self.offset_y =self.offset_y -0.1 * self.canvas.winfo_height()
    #     self.update_image()

    # def pan_up(self, event):
    #     #self.offset_y = min(0, self.offset_y +0.1 * self.canvas.winfo_height())
    #     self.offset_y = self.offset_y +0.1 * self.canvas.winfo_height()
    #     self.update_image()

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from sarpy.io.complex.sicd import SICDReader
from sarpy.visualization.remap import Density
import os

remap = Density()

class SARViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SAR Image Viewer")
        self.image_label = tk.Label(self.root)
        self.image_label.pack()
        self.current_image_index = None
        self.image_paths = []

        menu_bar = tk.Menu(root)
        file_menu = tk.Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open Image", command=self.open_image)
        file_menu.add_command(label="Exit", command=root.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)
        root.config(menu=menu_bar)

        self.root.bind("<Left>", self.prev_image)
        self.root.bind("<Right>", self.next_image)

        self.canvas = tk.Canvas(self.root, bg="white")
        self.canvas.pack(fill=tk.BOTH, expand=tk.YES)

        # Bind mouse events for zooming and panning
        self.canvas.bind("<ButtonPress-1>", self.pan_start)
        self.canvas.bind("<B1-Motion>", self.pan_move)
        self.canvas.bind("<MouseWheel>", self.zoom)

        # Initial zoom level and pan offset
        self.zoom_level = 1.0
        self.offset_x = 0
        self.offset_y = 0

    def zoom(self, event):
        factor = 1.1 if event.delta > 0 else 0.9
        self.zoom_level *= factor
        self.update_image()

    def pan_start(self, event):
        self.pan_x = event.x
        self.pan_y = event.y

    def pan_move(self, event):
        dx = event.x - self.pan_x
        dy = event.y - self.pan_y
        self.offset_x += dx
        self.offset_y += dy
        self.pan_x = event.x
        self.pan_y = event.y
        self.update_image()

    def update_image(self):
        new_width = int(self.image.width * self.zoom_level)
        new_height = int(self.image.height * self.zoom_level)
        resized_image = self.image.resize((new_width, new_height), Image.ANTIALIAS)
        self.image_tk = ImageTk.PhotoImage(resized_image)
        self.canvas.itemconfig(self.image_id, image=self.image_tk)
        self.canvas.configure(scrollregion=(0, 0, new_width, new_height))
        self.canvas.xview_moveto(self.offset_x / new_width)
        self.canvas.yview_moveto(self.offset_y / new_height)

    def open_image(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("SAR Images", "*.ntf *.nitf")])
        if file_paths:
            self.image_paths = list(file_paths)
            self.current_image_index = 0
            self.show_current_image()

    def show_current_image(self):
        if self.current_image_index is not None:
            file_path = self.image_paths[self.current_image_index]
            sar_image = SICDReader(file_path)[:]
            image = Image.fromarray(remap(sar_image))
            photo = ImageTk.PhotoImage(image)
            self.image_label.config(image=photo)
            self.image_label.image = photo

    def next_image(self, event):
        if self.current_image_index is not None:
            self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)
            self.show_current_image()

    def prev_image(self, event):
        if self.current_image_index is not None:
            self.current_image_index = (self.current_image_index - 1) % len(self.image_paths)
            self.show_current_image()

def main():
    root = tk.Tk()
    app = SARViewerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

    def update_image(self):
        if self.image:
            self.width = self.image.width * self.zoom_level
            self.height = self.image.height * self.zoom_level
            new_width = int(self.width)
            new_height = int(self.height)

            # Resize the image
            resized_image = self.image.resize((new_width, new_height), Image.LANCZOS)
            self.image_tk = ImageTk.PhotoImage(resized_image)

            # Update the image on canvas
            if self.image_id:
                self.canvas.itemconfig(self.image_id, image=self.image_tk)
                self.canvas.configure(scrollregion=(0, 0, new_width, new_height))
                self.canvas.xview_moveto(self.offset_x / new_width)
                self.canvas.yview_moveto(self.offset_y / new_height)

            # Update the coordinates of drawn rectangles
            for rect in self.rectangles:
                ratio = self.zoom_level/self.rect_zoom
                x1, y1, x2, y2 = self.canvas.coords(rect)
                # Calculate new coordinates based on zoom level
                x1_img = x1* ratio
                y1_img = y1* ratio
                x2_img = x2* ratio
                y2_img = y2* ratio
                # Update rectangle coordinates
                
                self.canvas.coords(rect, x1_img, y1_img, x2_img, y2_img)
            self.rect_zoom = self.zoom_level


In [ ]:
import os
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import filedialog, scrolledtext
from PIL import Image, ImageTk
from sarpy.geometry import point_projection
from sarpy.io.complex.sicd import SICDReader
from sarpy.visualization.remap import Density

remap = Density()

class SICDViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SICD Viewer")
        self.image_label = tk.Label(self.root)  # No need for this label
        self.image_label.pack()  # Remove this line
        self.current_image_index = None
        self.image_paths = []

        menu_bar = tk.Menu(root)
        file_menu = tk.Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open Image", command=self.open_image)
        file_menu.add_command(label="Exit", command=root.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)
        root.config(menu=menu_bar)

        # Add label for displaying file name
        self.text_box = scrolledtext.ScrolledText(self.root, wrap=tk.NONE, font=("Helvetica", 12), height=.5, width=150)
        self.text_box.pack(expand=False)

         # Bind arrow keys for navigation
        #self.root.bind("<Left>", lambda event=None: self.prev_image())
        #self.root.bind("<Right>", lambda event=None: self.next_image())

        self.button_frame = tk.Frame(root)
        self.button_frame.pack(side="top")

        # Create buttons for next and previous images
        self.prev_button = tk.Button(self.root, text="Prev", command=self.prev_image)
        self.prev_button.pack(side=tk.LEFT)
        self.next_button = tk.Button(self.root, text="Next", command=self.next_image)
        self.next_button.pack(side=tk.RIGHT)
        self.export_latlon_button = tk.Button(self.button_frame, text="Export Rectangles LatLon", command=self.export_latlon)
        self.export_latlon_button.pack(side=tk.RIGHT)
        self.export_pix_button = tk.Button(self.button_frame, text="Export Rectangles Pixels", command=self.export_pix)
        self.export_pix_button.pack(side=tk.LEFT)

        self.canvas = tk.Canvas(self.root, bg="black")
        self.canvas.pack(fill=tk.BOTH, expand=tk.YES)
        

        # Bind events for zooming and panning

        self.canvas.bind("<MouseWheel>", self.zoom)
        self.root.bind('<Left>', self.pan_left)
        self.root.bind('<Right>', self.pan_right)
        self.root.bind('<Up>', self.pan_up)
        self.root.bind('<Down>', self.pan_down)

        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonPress-3>", self.right_click)
        self.canvas.bind("<B1-Motion>", self.on_move_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

        # Initial zoom level and pan offset
        self.zoom_level = 1.0
        self.offset_x = 0
        self.offset_y = 0

        # Initialize image and image ID
        self.image = None
        self.image_id = None

        # Rectangle drawing variables
        self.csv = None
        self.rect = None
        self.start_x = None
        self.start_y = None
        # List to keep track of drawn rectangles
        self.rectangles = []
        
    def zoom(self, event):
        x = self.canvas.canvasx(event.x)
        y = self.canvas.canvasy(event.y)
        factor = 1.001**event.delta 
        self.canvas.scale(tk.ALL, x,y,factor,factor)

        # # Calculate the difference in coordinates before and after the zoom
        # delta_x = (x - self.canvas.canvasx(event.x)) * factor
        # delta_y = (y - self.canvas.canvasy(event.y)) * factor
        
        # # Adjust the scroll region of the canvas to keep the mouse cursor fixed
        # self.canvas.xview_scroll(int(delta_x), "units")
        # self.canvas.yview_scroll(int(delta_y), "units")

        # Adjust the scroll region of the canvas
        # new_width = int(self.canvas.winfo_width() * factor)
        # new_height = int(self.canvas.winfo_height() * factor)
        # new_x = x - (x - self.canvas.xview()[0] * self.canvas.winfo_width()) * factor
        # new_y = y - (y - self.canvas.yview()[0] * self.canvas.winfo_height()) * factor
        # self.canvas.configure(scrollregion=(0, 0, new_width, new_height))
        # self.canvas.xview_moveto(new_x / new_width)
        # self.canvas.yview_moveto(new_y / new_height)
        self.zoom_level *= factor
        
        self.update_image()


    # def zoom(self, event):
    #     cursor_x = self.canvas.canvasx(event.x)
    #     cursor_y = self.canvas.canvasy(event.y)
    #     factor = 1.1 if event.delta > 0 else 0.9
        
        
    #     # Adjust the offset based on the cursor position
    #     self.offset_x = cursor_x - factor * (cursor_x - self.offset_x)
    #     self.offset_y = cursor_y - factor * (cursor_y - self.offset_y)

    #     self.zoom_level *= factor
        
    #     self.update_image()


    def pan_start(self, event):
        self.pan_x = event.x
        self.pan_y = event.y
    

    def pan_left(self, event):
        #self.offset_x = max(0, self.offset_x -0.1 * self.canvas.winfo_width())
        self.offset_x =  self.offset_x -0.1 * self.canvas.winfo_width()
        self.update_image()

    def pan_right(self, event):
        #self.offset_x = min(self.canvas.winfo_width(), self.offset_x +0.1 * self.canvas.winfo_width())
        self.offset_x = self.offset_x +0.1 * self.canvas.winfo_width()
        self.update_image()

    def pan_down(self, event):
        #self.offset_y =max(-self.canvas.winfo_height(), self.offset_y -0.1 * self.canvas.winfo_height())
        self.offset_y =self.offset_y -0.1 * self.canvas.winfo_height()
        self.update_image()

    def pan_up(self, event):
        #self.offset_y = min(0, self.offset_y +0.1 * self.canvas.winfo_height())
        self.offset_y = self.offset_y +0.1 * self.canvas.winfo_height()
        self.update_image()
        
    
    def update_image(self):
        if self.image:
            self.width = self.image.width * self.zoom_level
            self.height = self.image.height * self.zoom_level
            new_width = int(self.width)
            new_height = int(self.height)

            # Resize the image
            resized_image = self.image.resize((new_width, new_height), Image.LANCZOS)
            self.image_tk = ImageTk.PhotoImage(resized_image)

            # Update the image on canvas
            if self.image_id:
                self.canvas.itemconfig(self.image_id, image=self.image_tk)
                self.canvas.configure(scrollregion=(0, 0, new_width, new_height))
                self.canvas.xview_moveto(self.offset_x / new_width)
                self.canvas.yview_moveto(self.offset_y / new_height)

            # # Update the coordinates of drawn rectangles
            # for rect in self.rectangles:
            #     ratio = self.zoom_level/self.rect_zoom
            #     x1, y1, x2, y2 = self.canvas.coords(rect)
            #     # Calculate new coordinates based on zoom level
            #     x1_img = x1* ratio
            #     y1_img = y1* ratio
            #     x2_img = x2* ratio
            #     y2_img = y2* ratio
            #     # Update rectangle coordinates
                
            #     self.canvas.coords(rect, x1_img, y1_img, x2_img, y2_img)
            # self.rect_zoom = self.zoom_level


    def open_image(self):
        self.clear_rect()
        file_paths = filedialog.askopenfilenames(filetypes=[("SAR Images", "*.ntf *.nitf"), ("Pictures", "*.jpg *.jpeg *.png")])
        if len(file_paths) >1:
            self.image_paths = list(file_paths)
            self.current_image_index = 0
            self.show_current_image()
        elif len(file_paths) ==1:
            self.directory = os.path.dirname(file_paths[0])
            self.image_paths = [f"{self.directory}/{file}" for file in os.listdir(self.directory) if os.path.isfile(os.path.join(self.directory, file))]
            self.current_image_index = self.image_paths.index(file_paths[0])
            self.show_current_image()


    def show_current_image(self):
        if self.current_image_index is not None:
            #self.text_box.config(state=tk.NORMAL)
            #self.text_box.delete(1.0, tk.END)
            #self.text_box.insert('1.0', f"Loading", "center")
            #self.text_box.tag_configure("center", justify="center")
            #self.text_box.config(state=tk.DISABLED) 

            file_path = self.image_paths[self.current_image_index]
            if file_path.endswith(('.ntf', '.nitf')):
                self.sicd = SICDReader(file_path)
                sar_image = self.sicd[:]
                self.image = Image.fromarray(remap(sar_image))
            elif file_path.endswith((".jpg", ".jpeg", ".png")):
                self.image = Image.open(file_path)
            

            # Calculate initial zoom level to fit the entire image in the canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            image_width, image_height = self.image.size
            zoom_x = canvas_width / image_width
            zoom_y = canvas_height / image_height
            self.zoom_level = min(zoom_x, zoom_y)

            self.image_tk = ImageTk.PhotoImage(self.image.resize((int(image_width * self.zoom_level),
                                                                int(image_height * self.zoom_level))))
            if self.image_id:
                self.canvas.delete(self.image_id)  # Remove previous image from canvas
            self.image_id = self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_tk)
            self.update_image()
            
            self.text_box.config(state=tk.NORMAL)
            self.text_box.delete(1.0, tk.END)
            self.text_box.insert('1.0', f"{file_path}\n", "center")
            self.text_box.tag_configure("center", justify="center")


    def next_image(self):    
        self.clear_rect()
        if self.current_image_index is not None:
            self.current_image_index = (self.current_image_index + 1)% len(self.image_paths)
            self.show_current_image()

    def prev_image(self):
        self.clear_rect()
        if self.current_image_index is not None:
            self.current_image_index = (self.current_image_index - 1)% len(self.image_paths)
            self.show_current_image()

    def on_button_press(self, event):
        # create rectangle if not yet exist
        x = self.canvas.canvasx(event.x)
        y = self.canvas.canvasy(event.y)
        self.start_x = x
        self.start_y = y
        self.rect = self.canvas.create_rectangle(x, y, x, y, fill="", outline="red")
        self.rectangles.append(self.rect)

    def on_move_press(self, event):
        # expand rectangle as you drag the mouse
        x = self.canvas.canvasx(event.x)
        y = self.canvas.canvasy(event.y)
        self.canvas.coords(self.rect, self.start_x, self.start_y, x, y)

    def on_button_release(self, event):
        # clear the rectangle
        self.rect_zoom = self.zoom_level
        self.rect = None

    def right_click(self,event):
        if self.rectangles:
            self.canvas.delete(self.rectangles.pop())

    def clear_rect(self):
        while self.rectangles:
            self.canvas.delete(self.rectangles.pop())

    def export_pix(self):
        x, y = self.width, self.height        
        w, h = self.image.width, self.image.height
        for rect in self.rectangles:
            x1, y1, x2, y2 = self.canvas.coords(rect)
            print([w*x1/x, h*y1/y],[w*x2/x, h*y2/y])
        with open('pixels.txt', "w") as f:
            for rect in self.rectangles:
                x1, y1, x2, y2 = self.canvas.coords(rect)
                out0 = (x1/x+x2/x)/2
                out1 = (y1/y + y2/y)/2
                out2 = (x2/x - x1/x)
                out3 = (y2/y-y1/y)
                print(out0, out1,out2,out3)
                # print(x1/x *w , y1/x * h, x2/x *w , h*y2/y)
                # print("hey")
                f.write(f"0 {str(out0)} {str(out1)} {str(out2)} {str(out3)} \n")


    def export_latlon(self):
        lat0,lon0 = self.sicd.sicd_meta.GeoData.ImageCorners.FRFC
        lat,lon = self.sicd.sicd_meta.GeoData.ImageCorners.LRLC -self.sicd.sicd_meta.GeoData.ImageCorners.FRFC
        #print(self.sicd.sicd_meta.GeoData.ImageCorners.LRLC -self.sicd.sicd_meta.GeoData.ImageCorners.FRFC, self.sicd.sicd_meta.GeoData.ImageCorners.LRFC -self.sicd.sicd_meta.GeoData.ImageCorners.FRLC)
        print(self.sicd.sicd_meta.GeoData.ImageCorners.FRFC,self.sicd.sicd_meta.GeoData.ImageCorners.FRLC,self.sicd.sicd_meta.GeoData.ImageCorners.LRLC,self.sicd.sicd_meta.GeoData.ImageCorners.LRFC)
        x, y = self.width, self.height
        w, h = self.image.width, self.image.height
        def xy_latlon(xy):
            # coords = point_projection.image_to_ground_geo(xy, self.sicd.sicd_meta, projection_type='PLANE')
            # return coords[:2]
            return xy[:2]
        columns = ['center', "corner0","corner1", "corner2", "corner3"]
        column_names = ["name"]
        column_names.extend([f"{column}_{latlon}" for column in columns for latlon in ['x', 'y']])
        # column_names.extend([f"{column}_{latlon}" for column in columns for latlon in ['lat', 'lon']])
        csv = os.path.join(self.directory,"bounding_box.csv")
        if os.path.isfile(csv):
            latlonDF = pd.read_csv(csv)
        else:
            latlonDF = pd.DataFrame(columns=column_names)
        for rect in self.rectangles:
            x1, y1, x2, y2 = self.canvas.coords(rect)
            print(x,y, w,h)
            print([w*x1/x, h*y1/y],[w*x2/x, h*y2/y])
            print(xy_latlon([w*x1/x, h*y1/y]),xy_latlon([w*x2/x, h*y2/y]))
            latlon_arr = np.concatenate([[os.path.basename(self.image_paths[self.current_image_index])],
                                           xy_latlon([w/x*(x1+x2)/2, h/y*(y1+y2)/2]),
                                           xy_latlon([w*x1/x, h*y1/y]),xy_latlon([w*x2/x, h*y1/y]),
                                           xy_latlon([w*x2/x, h*y2/y]),xy_latlon([w*x1/x, h*y2/y])])
            latlonDF.loc[len(latlonDF)] = latlon_arr
        latlonDF.drop_duplicates(keep='first', inplace=True)
        latlonDF.to_csv(csv, index=False)

        
root = tk.Tk()
root.state('zoomed')
app = SICDViewerApp(root)
app.open_image()
# Set focus to the canvas after a short delay
root.after(100, root.focus_force)
root.mainloop()



[46.54007194 10.40621881] [46.55651617 10.45612765] [46.51350558 10.4839403 ] [46.49769462 10.43366767]
657.4920592193808 899.0 5434 7430
[2735.6284760845388, 661.1790878754172] [4628.25361512792, 2644.7163515016687]
[2735.6284760845388, 661.1790878754172] [4628.25361512792, 2644.7163515016687]
657.4920592193808 899.0 5434 7430
[314.0600667408232, 2099.2436040044495] [2082.7141268075643, 4867.931034482759]
[314.0600667408232, 2099.2436040044495] [2082.7141268075643, 4867.931034482759]
657.4920592193808 899.0 5434 7430
[1570.3003337041157, 3206.718576195773] [3743.92658509455, 5157.196885428254]
[1570.3003337041157, 3206.718576195773] [3743.92658509455, 5157.196885428254]
657.4920592193808 899.0 5434 7430
[3719.1323692992214, 4405.105672969967] [4479.488320355951, 5157.196885428254]
[3719.1323692992214, 4405.105672969967] [4479.488320355951, 5157.196885428254]
657.4920592193808 899.0 5434 7430
[4305.928809788655, 3595.1612903225805] [5124.137931034483, 4867.931034482759]
[4305.928809788

In [ ]:
def pan_left(self, event):
        print(self.offset_x, self.offset_y)
        self.offset_x = max(0, self.offset_x -0.1 * self.canvas.winfo_width())
        self.update_image()

    def pan_right(self, event):
        print(self.offset_x, self.offset_y)
        self.offset_x = min(self.canvas.winfo_width(), self.offset_x +0.1 * self.canvas.winfo_width())
        self.update_image()

    def pan_down(self, event):
        print(self.offset_x, self.offset_y)
        self.offset_y =min(self.canvas.winfo_height(), self.offset_y +0.1 * self.canvas.winfo_height())
        self.update_image()

    def pan_up(self, event):
        print(self.offset_x, self.offset_y)
        self.offset_y = max(0, self.offset_y -0.1 * self.canvas.winfo_height())
        self.update_image()